<a href="https://colab.research.google.com/github/deniseiras/EscolaVeraoSantosDumont2023/blob/main/taxi_cab_2021_map_JCD_MC_CD02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Yellow Taxi cab: registra viagens de taxi em Manhattan
![picture](https://drive.google.com/uc?export=view&id=1hoFUFSm5IJG-5CQYBtqDK_e2MvMODc1P)
## Este notebook explora o processamento de dados tabulares (i.e viagens de taxi)
## Neste exemplo, usamos a API RDD.
## Procuramos por viagens não muito vantajosas, como muitos passageiros e baixa tarifa :)

##site: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 39.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=d7fda5c5e0fa993faeb17867966f5986710cfe75550cd68624959b62f2dc65f7
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
from pyspark import SparkContext
import math

In [ ]:
from google.colab import drive

## Monta um google drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cd /content/gdrive/
!ls -la gdrive/MyDrive/BigData/data
!head gdrive/MyDrive/BigData/data/yellow_tripdata_2021-01.csv


total 123029
-rw------- 1 root root 125981363 Feb 11 17:33 yellow_tripdata_2021-01.csv
VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,2021-01-01 00:30:10,2021-01-01 00:36:12,1,2.10,1,N,142,43,2,8,3,0.5,0,0,0.3,11.8,2.5
1,2021-01-01 00:51:20,2021-01-01 00:52:19,1,.20,1,N,238,151,2,3,0.5,0.5,0,0,0.3,4.3,0
1,2021-01-01 00:43:30,2021-01-01 01:11:06,1,14.70,1,N,132,165,1,42,0.5,0.5,8.65,0,0.3,51.95,0
1,2021-01-01 00:15:48,2021-01-01 00:31:01,0,10.60,1,N,138,132,1,29,0.5,0.5,6.05,0,0.3,36.35,0
2,2021-01-01 00:31:49,2021-01-01 00:48:21,1,4.94,1,N,68,33,1,16.5,0.5,0.5,4.06,0,0.3,24.36,2.5
1,2021-01-01 00:16:29,2021-01-01 00:24:30,1,1.60,1,N,224,68,1,8,3,0.5,2.35,0,0.3,14.15,2.5
1,2021-01-01 00:00:28,2021-01-01 00:17:28,1,4.10,1,N,95,157,2,16,0.5,0.5,0,0,0.3,17.3,0
1,2021-01-01 00:12:29

In [ ]:
spark = SparkContext(appName="aula-1", master="local[4]")

## Lê o arquivo no google drive
- O resultado cria um RDD com uma entrada por linha do arquivo
- um filtro retira a linha de cabeçalho:
VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge

In [ ]:
taxi_file_header = spark.textFile("/content/gdrive/MyDrive/BigData/data/yellow_tripdata_2021-01.csv")
taxi_file= taxi_file_header.filter(lambda l: not l.startswith("VendorID"))

In [ ]:
def testa_VendorId(id):
  if not id.isnumeric():
    return False
  else:  
    if (int(id) <= 20):
      return True
    else:
      return False    

## Gera um RDD onde cada item é extraido do arquivo a partir
## da delimitação por vírgula.
## Em seguida, filtra as entradas com VendorId <= 20

In [ ]:
print(taxi_file.count())
trip_column= taxi_file.map(lambda a: a.split(","))
print(trip_column.take(1))
list_trip = trip_column.filter(lambda a:testa_VendorId(a[0]))

1369765
[['1', '2021-01-01 00:30:10', '2021-01-01 00:36:12', '1', '2.10', '1', 'N', '142', '43', '2', '8', '3', '0.5', '0', '0', '0.3', '11.8', '2.5']]


In [ ]:

print("Total de Linhas:"+str(list_trip.count()))

Total de Linhas:1271413


## Criando pipelines sobre o RDD list_trip

# Spark Dataflow

1.   map
2.   groupBy
3.   MapValues








In [ ]:

transformIntRDD = list_trip.map(lambda a: (a[0],float(a[11])))
transformRDD = transformIntRDD.groupByKey().mapValues(lambda v:sum(v))

In [ ]:
transformRDD.take(2)

[('1', 1027690.61), ('2', 221895.09999999992)]


# Projeta: VendorId, Numero de passageiros, Valores de Tarifa 



In [ ]:
projectedRDD = list_trip.map(lambda a: (a[0],a[3],a[11]))




## DataFlow: Calcula viagens prejuizo



1.   filter
2.   map





In [ ]:
print(projectedRDD.take(10))
print(projectedRDD.count())
tarifa_altaRDD = projectedRDD.filter(lambda a: float(a[2]) >5 and int(a[1])>3)
print(tarifa_altaRDD.count())

[('1', '1', '3'), ('1', '1', '0.5'), ('1', '1', '0.5'), ('1', '0', '0.5'), ('2', '1', '0.5'), ('1', '1', '3'), ('1', '1', '0.5'), ('1', '1', '3'), ('1', '1', '0.5'), ('1', '2', '3')]
1271413
9


In [ ]:
prejuizo=tarifa_altaRDD.count()

In [ ]:
print("Viagens Prejuizo: "+str(prejuizo))

Viagens Prejuizo: 9
